# Implement HiTS-EQ in python - both Equation and curve fitting solutions
Hsuan-Chun Lin 2023.03

In this version, multiple points are supported. You can use different number of concentrations without change any parameters in the sourcecode.
## Curve Fitting Solution - recommended
### Import necessary packages

In [131]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from functools import partial
import seaborn as sns

### Import data in csv format and fetch the binding fractions

In [132]:
df = pd.read_csv("input_data.csv", header=0)
df.head()

,Seq,1,2,3,4
0,f,0,0.25,0.5,0.75
1,[E],0,6.67,20.0,60.00
2,AAAAAA,4133,1425.00,1754.0,1422.00
3,AAAAAC,2662,1049.00,998.0,953.00
4,AAAAAG,5247,2726.00,2612.0,2322.00


### Data format preprocessing

In [149]:
enzyme_concentration =  df.iloc[1,1:]
binding_fraction = 1 - df.iloc[0,1:].astype(float)
sequence_count = df.iloc[2:,1:]
library = df.iloc[2:,0].reset_index().drop("index",axis=1)

### Define fitting function
$$f = 1 -( \frac{D_{t}}{D_t1}\times (1-frac) \times \frac{D_1}{D_0})= \frac{E}{E+K_D}$$

Calculate $$ D_t $$ and $$D_{t1}$$

In [134]:
Dt_all = sequence_count.sum()
Dt_Dt1_all = Dt_all[0]/Dt_all
Dt_Dt1_all


1    1.000000
2    1.797373
3    1.732612
4    1.686968
dtype: float64

In [141]:
D0_ = sequence_count.iloc[:,0]
D1fD0 = sequence_count.divide(D0_,axis='rows')
fitting_table = 1 - (Dt_Dt1_all * binding_fraction * D1fD0)
fitting_table[fitting_table < 0] = 0.00001
fitting_table = fitting_table.reset_index()
fitting_table = fitting_table.drop("index",axis = 1)

### Define fitting equations and implement it by curve_fit and partial functions

In [142]:

def HitsEQ(x, K):
    return x/(x+K)

def curvefit(func, x, y):
    return tuple(curve_fit(HitsEQ, x, y))
fit = partial(curvefit, HitsEQ, enzyme_concentration)



### Fit your data and get results

In [143]:
K = fitting_table.apply(fit, axis=1).apply(pd.Series)


In [179]:
K.columns = ["K", "pcov"]
finalresult = K.explode(["K", "pcov"]).explode("pcov").reset_index().drop("index", axis = 1)
finalresult[["id"]] = library
finalresult[["KA"]] = (1/finalresult.K).apply(pd.Series)
finalresult.head()

,K,pcov,id,KA
0,7.96889,2.596841,AAAAAA,0.125488
1,8.600381,0.477099,AAAAAC,0.116274
2,15.0288,0.144638,AAAAAG,0.066539
3,6.204218,0.366085,AAAAAT,0.161181
4,4.82366,0.054333,AAAACA,0.207311


### Calculate relative association constants

In [180]:
finalresult[["RKA"]] = (finalresult.KA/finalresult.KA[2]).apply(pd.Series)

,K,pcov,id,KA,RKA
0,7.96889,2.596841,AAAAAA,0.125488,1.885934
1,8.600381,0.477099,AAAAAC,0.116274,1.747457
2,15.0288,0.144638,AAAAAG,0.066539,1.000000
3,6.204218,0.366085,AAAAAT,0.161181,2.422352
4,4.82366,0.054333,AAAACA,0.207311,3.115642


In [189]:
cols = ['id', 'K', 'KA', 'RKA', 'pcov']
finalresult = finalresult[cols]
finalresult.head()

,id,K,KA,RKA,pcov
0,AAAAAA,7.96889,0.125488,1.885934,2.596841
1,AAAAAC,8.600381,0.116274,1.747457,0.477099
2,AAAAAG,15.0288,0.066539,1.000000,0.144638
3,AAAAAT,6.204218,0.161181,2.422352,0.366085
4,AAAACA,4.82366,0.207311,3.115642,0.054333


## Equation solution - use unbound and one fraction point

In [ ]:
def HitsEQ_calculation(x,y):
    return 